In [0]:
# Creating the Title_df DataFrame from the bronze_title_ratings table
title_df = spark.table('az_adb_simbus_training.imdb_project_schema.bronze_title_ratings')
title_df.limit(5).display()

In [0]:
# Creating the basics_df DataFrame from the bronze_title_basics table
basics_df = spark.table('az_adb_simbus_training.imdb_project_schema.bronze_title_basics')
basics_df.limit(5).display()

In [0]:
# Performing an inner join between the title_df and title_ratings tables.
joined_df = title_df.join(basics_df,title_df.tconst == basics_df.tconst, "inner")
joined_df.limit(5).display()


In [0]:
#Checking the distinct values of the titleType column
joined_df.select('titleType').distinct().show()

In [0]:
# Filter out only movies (titleType == 'movie'). 
# Extract year, genres, average_Rating, num_Votes. 
from pyspark.sql.functions import col 
selected_df = (joined_df.filter(col('titleType') == 'movie')               
                .select('startYear','endYear','genres','averageRating', 'numVotes', 'titleType'))
selected_df.limit(10).display()                       
                            
                             

In [0]:
# Created the silver_UseCase1_table table in the imdb_project_schema schema from the above filtration.
selected_df.write.mode('overwrite').saveAsTable('az_adb_simbus_training.imdb_project_schema.silver_UseCase1_table')


In [0]:
%sql
select * from az_adb_simbus_training.imdb_project_schema.silver_usecase1_table limit 5;